In [1]:
import cv2
import numpy as np

import ipywidgets as widgets
from IPython.display import display, clear_output
from functools import partial
import time

from tiki.mini import TikiMini
from testing import capture_generator

In [2]:
tiki = TikiMini()

In [3]:
# Hyper Parameters
# 실전용
#lower_event = np.array([67, 23, 101])
#upper_event = np.array([80, 255, 168])

# 연습용
#lower_event = np.array([41, 57, 50])
#upper_event = np.array([52, 153, 161])

 #Byung Ddu ggung
lower_event = np.array([90, 100, 100])
upper_event = np.array([150, 255, 255])

#절연테이프
lower_line = np.array([0, 0, 47])
upper_line = np.array([179,147,110])

#연습용, 실전용
#lower_line = np.array([0, 0, 0])
#pper_line = np.array([220, 70, 130])

roix_s = 280
roix_e = 350
roiy_s = 120
roiy_e = 560

In [4]:
def stop():
    tiki.stop()

def mission():
    tiki.set_motor_power("MOTOR_LEFT", 0)
    tiki.set_motor_power("MOTOR_RIGHT", 0)

def turn_90_degrees(direction):
    turn_duration = 1.5
    turn_power1 = 15
    turn_power2 = 15

    if direction == 'left':
        tiki.counter_clockwise(turn_power1)
    elif direction == 'right':
        tiki.clockwise(turn_power2)
    
    time.sleep(turn_duration)
    tiki.stop()

def turn_90(direction):
    turn_power1 = 15
    turn_power2 = 15

    if direction == 'left':
        tiki.counter_clockwise(turn_power1)
    elif direction == 'right':
        tiki.clockwise(turn_power2)

In [5]:
class Wrapper:
    def __init__(self, save_name=None):
        self.cap = cv2.VideoCapture(
            "nvarguscamerasrc ! video/x-raw(memory:NVMM), width=640, height=480, framerate=15/1, format=NV12 ! "
            "nvvidconv flip-method=2 ! video/x-raw, format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink"
        )
    def __enter__(self):
        print('entered')
        return self.cap
    def __exit__(self,a,b,c):
        self.cap.release()
        print('released')

In [6]:
def detect_lines(image):
    
    crop_img = image[roix_s:roix_e, roiy_s:roiy_e].copy()
    
    hsv_image = cv2.cvtColor(crop_img, cv2.COLOR_BGR2HSV)
    color_mask = cv2.inRange(hsv_image, lower_line, upper_line)
    masked_image = cv2.bitwise_and(crop_img, crop_img, mask=color_mask)
    gray_mask = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(gray_mask, 1, 255, cv2.THRESH_BINARY)
    lines, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    return crop_img, lines

In [7]:
def adjust_wheel_speed(cx, start_cx, event_num):
    black_list = [0]
    # CURVE
    if (cx > start_cx + 70):
        tiki.set_motor_power(tiki.MOTOR_LEFT, 50)
        tiki.set_motor_power(tiki.MOTOR_RIGHT, 10)
    elif (cx < start_cx - 70):
        tiki.set_motor_power(tiki.MOTOR_LEFT, 10)
        tiki.set_motor_power(tiki.MOTOR_RIGHT, 50)
    
    # MICRO CONTROL
    elif (cx > start_cx + 30):
        tiki.set_motor_power(tiki.MOTOR_LEFT, 50)
        tiki.set_motor_power(tiki.MOTOR_RIGHT, 30)
    elif (cx < start_cx - 30):
        tiki.set_motor_power(tiki.MOTOR_LEFT, 30)
        tiki.set_motor_power(tiki.MOTOR_RIGHT, 50)
        
    # NORMAL
    elif (start_cx - 30 < cx < start_cx + 30): 
        tiki.set_motor_power(tiki.MOTOR_LEFT, 30)
        tiki.set_motor_power(tiki.MOTOR_RIGHT, 30) 
    return

In [8]:
def detect_event_color(image, lower_event, upper_event):
    
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_event, upper_event)
    return mask

def calculate_center_area(image, center_width=200, center_height=50):
    height, width = image.shape[:2]
    
    center_x = width // 2 
    center_y = height // 2 + 10
    start_x = center_x - center_width // 2
    start_y = center_y - center_height // 2
    end_x = start_x + center_width
    end_y = start_y + center_height

    return (start_x, start_y, end_x, end_y)

def detect_event_in_center(center_mask, start_x, start_y):
    event_detected = np.sum(center_mask) > 0
    
    event_point = None
    if event_detected:
        contours, _ = cv2.findContours(center_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            M = cv2.moments(largest_contour)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"]) + start_x
                cy = int(M["m01"] / M["m00"]) + start_y
                event_point = (cx, cy)
    
    return event_detected, event_point

In [9]:
def add_callback(callbacks,callback, end_time, argu):
    callback = partial(callback, argu)
    callbacks.append((callback, time.time() + end_time))

In [10]:
def Event_Handler(callbacks, event_num):
    Time90 = 1.5
    if event_num == 0:
        add_callback(callbacks, turn_90, Time90, 'left')
        return 20
    elif event_num == 1: # mission
        add_callback(callbacks, turn_90, Time90, 'left')
        
        # Mission
        add_callback(callbacks,tiki.counter_clockwise, 3.5, 10)
        add_callback(callbacks,tiki.clockwise, 7.5, 10)
        add_callback(callbacks,tiki.counter_clockwise, 9.5, 10)
        
        add_callback(callbacks, turn_90, 11, 'left')
        return 15
    elif event_num == 2:
        add_callback(callbacks, turn_90, Time90, 'left')
        return 40
    elif event_num == 3: # mission
        add_callback(callbacks, turn_90, Time90, 'left')
        mission()
        add_callback(callbacks, turn_90, 3, 'right')
        return 40
    elif event_num == 4:
        add_callback(callbacks, turn_90, Time90, 'right')
        return 20
    elif event_num == 5: # mission
        add_callback(callbacks, turn_90, Time90, 'left')
        # Mission
        add_callback(callbacks, turn_90, 3, 'right')
        return 80
    elif event_num == 6: # mission
        add_callback(callbacks, turn_90, Time90, 'left')
        # Mission
        add_callback(callbacks, turn_90, 3, 'right')
        return 40
    elif event_num == 7:
        add_callback(callbacks, turn_90, Time90, 'left')
        return 20
    elif event_num == 8:
        stop()
    

In [11]:
def main():
    video_widget = widgets.Image(format='bmp')
    
    tiki.set_motor_mode(tiki.MOTOR_MODE_PID)
    event_detection_count = 0
    event_detection_threshold = 3
    
    event_cnt = 0
    
    cool_flag = 1
    cool_time = 0
    cx = 0
    start_cx = None
    start_time = time.time()  # 시작 시간 기록

    frametimes = []
    cnt = 0
    t_prev = time.time()
    callbacks = []
    
    clear_output(wait=True)
    display(video_widget)

    for frame in capture_generator():
        t_cur = time.time()
        for callback, end_time in callbacks:
            if t_cur <= end_time:
                callback()
                break
        callbacks = list(filter(lambda c_t: c_t[1] > t_cur, callbacks))

        # Line
        crop_img, lines = detect_lines(frame)
        
        if len(lines) > 0:
            c = max(lines, key=cv2.contourArea)
            M = cv2.moments(c)
            if M['m00'] != 0:
                cx = int(M['m10']/M['m00'])
                cy = int(M['m01']/M['m00'])

                # 3초 후의 cx 값을 start_cx로 설정
                if start_cx is None and t_cur - start_time >= 3:
                    start_cx = cx  

        cv2.drawContours(crop_img, lines, -1, (0,255,0), 1)
        if len(callbacks) == 0:
            if start_cx is not None:
                adjust_wheel_speed(cx, start_cx, event_cnt)
            
            # Event Detection
            start_x, start_y, end_x, end_y = calculate_center_area(frame)
            center = frame[start_y:end_y, start_x:end_x]
            event_mask = detect_event_color(center, lower_event, upper_event)
            event_detected, event_point = detect_event_in_center(event_mask, start_x, start_y)
            
            if event_detected:
                event_detection_count += 1
            else:
                event_detection_count = 0
    
            cv2.rectangle(frame, (start_x, start_y), (end_x, end_y), (0, 255, 255), 2)
    
            if event_point:
                cv2.circle(frame, event_point, 10, (0, 255, 0), -1)
                cv2.putText(frame, f"Green: ({event_point[0]}, {event_point[1]})", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
            # Event 
            if cool_time <= 0:
                cool_flag = 1
            if(event_detection_count >= event_detection_threshold and cool_flag): 
                cool_time = Event_Handler(callbacks, event_cnt)
                tiki.stop()
                event_cnt += 1
                cool_flag = 0
            if cool_time > 0:
                cool_time -= 1
        
        frame[roix_s:roix_e, roiy_s:roiy_e] = crop_img

        cv2.putText(frame, f"Current Event: {event_cnt}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        cv2.putText(frame, f"cx: {cx}s", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

        cv2.putText(frame, f"Cool Time: {cool_time}s", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        cv2.putText(frame, f"Cool Flag: {cool_flag}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

        _, buffer = cv2.imencode('.bmp', frame)
        video_widget.value = buffer.tobytes()
        
        # clear_output(wait=True)
        # time.sleep(0.1)  # 프레임 간 간격

        
        t_cur = time.time()
        frametimes.append(t_cur - t_prev)
        t_prev = t_cur
        cnt += 1
        if cnt % 15 == 0:
            #print(f'avg frame: {1 / np.mean(frametimes):.2f} fps')
            frametimes.clear()

if __name__ == '__main__':
    main()

Image(value=b'', format='bmp')

GST_ARGUS: Creating output stream
CONSUMER: Waiting until producer is connected...
GST_ARGUS: Available Sensor modes :
GST_ARGUS: 3264 x 2464 FR = 21.000000 fps Duration = 47619048 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 3264 x 1848 FR = 28.000001 fps Duration = 35714284 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1920 x 1080 FR = 29.999999 fps Duration = 33333334 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1640 x 1232 FR = 29.999999 fps Duration = 33333334 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1280 x 720 FR = 59.999999 fps Duration = 16666667 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1280 x 720 FR = 120.000005 fps Duration = 8333333 ; Analog Gain range min 1.000000, max 10.625000; Exposur

[ WARN:0@1.255] global cap_gstreamer.cpp:1777 open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


released


KeyboardInterrupt: 